In [1]:
import pandas as pd

In [20]:
comments = pd.read_csv('Data/comments.csv')
fields = pd.read_csv('Data/fields.csv')

# swap FieldCode and FieldName column names
fields = fields.rename(columns={'FieldCode': 'FieldName', 'FieldName': 'FieldCode'}) # This line can be removed once the SQL script bug is fixed

# Filter
fields = fields[fields['Formula'].isnull() & fields['IsInSTATS'] == 1].drop(columns=['Formula', 'IsInSTATS']).sort_values(by=['UnitCode', 'FieldCode']).reset_index(drop=True)
# turn Date to datetime
comments['Date'] = pd.to_datetime(comments['Date'])
comments = comments.sort_values(by=['Date', 'UnitCode'], ascending=[True, True]).reset_index(drop=True)
comments['Year'] = comments['Date'].dt.year
comments['MonthName'] = comments['Date'].dt.month_name()
comments['MonthNumber'] = comments['Date'].dt.month

display(comments)
display(fields)

,UnitCode,ParkName,Date,Comments,Year,MonthName,MonthNumber
0,WORI,Women's Rights NHP,1993-01-01,Closed New Year's Day Two days at zero,1993,January,1
1,WORI,Women's Rights NHP,1993-02-01,Closed 2/13 at 4p.m. due to Blizzard Two days ...,1993,February,2
2,WORI,Women's Rights NHP,1993-03-01,Park Closed from 12:00 on 3/13 to 3/15 due to ...,1993,March,3
3,WORI,Women's Rights NHP,1993-07-01,Grand Opening of new Wesleyan Chapel Block hel...,1993,July,7
4,WORI,Women's Rights NHP,1993-08-01,Grand Opening of new Wesleyan Chapel Block hel...,1993,August,8
...,...,...,...,...,...,...,...
34857,WORI,Women's Rights NHP,2024-03-01,Visitor Center and Chapel are closed on Sunday...,2024,March,3
34858,WUPA,Wupatki NM,2024-03-01,3/15/24 - Closed due to inclement weather. 3/3...,2024,March,3
34859,ZION,Zion NP,2024-03-01,South Campground closed for construction until...,2024,March,3
34860,CHRI,Christiansted NHS,2024-04-01,on site education 1. 4/5 PR JROTC Cadets CHRI ...,2024,April,4


,ParkName,UnitCode,FieldCode,FieldName
0,Abraham Lincoln Birthplace NHP,ABLI,F13,ENDING TRAFFIC COUNT AT KNOB CREEK
1,Abraham Lincoln Birthplace NHP,ABLI,F2,ENDING TRAFFIC COUNT AT THE MAIN GATE
2,Abraham Lincoln Birthplace NHP,ABLI,F4,ENDING TRAFFIC COUNT AT THE PICNIC AREA
3,Abraham Lincoln Birthplace NHP,ABLI,F5,BUSES
4,Abraham Lincoln Birthplace NHP,ABLI,F6,BUS VISITORS
...,...,...,...,...
5543,Zion NP,ZION,F5,BUSES AT EAST ENTRANCE
5544,Zion NP,ZION,F6,BUS PASSENGERS AT EAST ENTRANCE
5545,Zion NP,ZION,F7,TRAFFIC COUNT AT KOLOB CANYON
5546,Zion NP,ZION,F8,BUSES AT KOLOB CANYON


In [15]:
# sum the number of words in each comment for the entire dataset
total_words = comments['Comments'].apply(lambda x: len(x.split())).sum()
print(total_words)
comments['Comments'].apply(lambda x: len(x.split()))

957633


0          8
1         11
2         12
3         15
4         22
        ... 
34857    176
34858     25
34859     19
34860     45
34861     14
Name: Comments, Length: 34862, dtype: int64

In [14]:
def getComments(unitCode: str = None, year: int = None):
    if unitCode is None or year is None:
        raise ValueError('Both unitCode and year must be provided')
    return comments[(comments['UnitCode'] == unitCode) & (comments['Year'] == year)]
def formatComments(unitCode: str = None, year: int = None):
    data = getComments(unitCode, year)
    if data.empty:
        return 'No comments found'
    # concatenate all comments into one string, the format should be for each month:
    """
    January:
    <comment1>

    February:
    <comment2>
    ...
    """

    formatted = ''
    for month in data['MonthName']:
        formatted += f"{month}:\n"
        formatted += data[data['MonthName'] == month]['Comments'].values[0] + '\n\n'
    return formatted
def getFieldNames(unitCode: str = None):
    if unitCode is None:
        raise ValueError('unitCode must be provided')
    return fields[fields['UnitCode'] == unitCode]['FieldName'].values
def createPrompt(unitCode: str = None, year: int = None):
    fields = getFieldNames(unitCode)
    comments = formatComments(unitCode, year)
    return f"Comments:\n{comments}\nOBJECTIVE: Review the list of comments that National Park data collectors left to describe the visitation at their park for each given month. Identify any months where specific locations were reported as closed to the public. If no closures are mentioned, please list 'No Closures'. Ignore places where the average or estimate is used. Do not include closures that were less than a few days or temporary closures due to weather or holidays. If you are unsure, default to no closures.\n\nINSTRUCTIONS: If a location is mentioned as closed in any month, please list it in the format:\nClosures:\n<Location>: <Month1>, <Month2>, ... <MonthN>.\n\n"

print(createPrompt('ROMO', 2023))

Comments:
January:
January was very wintery in RMNP, with lots of snow, wind, and cold temperatures. Weather affects visitation. The Colorado Vehicle Traction Law became active several times, based on weather and road conditions.  The Fall River Entrance is undergoing major construction this winter. Although the entrance is open, visitors are being advised to enter the Beaver Meadows Entrance. Due to this major construction project, Fall River Entrance is an estimate based on 40% of the Beaver Meadows Entrance January count.  Grand Lake Entrance is a 5-year average due to damage from the 2020 East Troublesome Wildfire, and Wild Basin Entrance is a 5-year average due to a broken counter.

February:
March was very wintery, including cold, cloudy, gray days and it was very windy. It also snowed several days, which is typical. Snow drifted deeply in high mountain areas due to the high winds. Due to a construction project the Fall River Entrance is open but traffic is being directed to the 

In [101]:
print(createPrompt('AMIS', 2022))

Field Names: ['TRAFFIC COUNT AT PECOS RIVER' 'TRAFFIC COUNT AT ROUGH CANYON ROAD'
 'Visitors at Visitor Center' 'TRAFFIC COUNT AT DIABLO EAST'
 'TRAFFIC COUNT AT BOX CANYON' 'TENTS AT GOVERNORS LANDING'
 'RVs AT GOVERNORS LANDING' 'TENTS AT SAN PEDRO' 'RVs AT SAN PEDRO'
 'TRAFFIC COUNT AT BLACK BRUSH POINT' 'TENTS AT SPUR 406'
 'RVs AT SPUR 406' 'TENTS AT 277 NORTH' 'RVs AT 277 NORTH'
 'TENTS AT 277 SOUTH' 'RVs AT 277 SOUTH' 'TENTS AT A.F. MARINA'
 'RVs AT A.F. MARINA' 'NPS BACKCOUNTRY: VARIOUS LOCATIONS (BOAT IN)'
 'GROUP OVERNIGHTS AT ROCK QUARRY CAMPGROUND' 'TRAFFIC COUNT AT SPUR 406'
 'GROUP OVERNIGHTS AT SAN PEDRO FLAT CAMPGROUND'
 'GROUP OVERNIGHTS AT 277 NORTH CAMPGROUND' 'TRAFFIC COUNT AT SAN PEDRO'
 'TRAFFIC COUNT AT SPUR 454' 'Researcher overnight stays'
 'Researchers in park' 'TRAFFIC COUNT AT GOVERNORS LANDING'
 'TRAFFIC COUNT AT U.S.A.F. MARINA' 'TRAFFIC COUNT AT OLD 277 NORTH']

Comments:
January:
Spur 406 counters is still out of service, Black Brush remains closed, San 

In [99]:
def processComments(unitCode: str = None, year: int = None):
    # get the comments for the given unitCode and year
    data = getComments(unitCode, year)
    if data.empty:
        return 'No comments found'
    # get the field names for the given unitCode
    fields = getFieldNames(unitCode)
    for comment in data['Comments']:
        # replace "  " with "\n" to make the comment more readable
        comment = comment.replace("  ", "\n\n")
        divider = '-' * 60
        prompt = f"COMMENTS:\n['{comment}']\n\n{divider}\n\nINSTRUCTIONS:\nListed Above are comments over the year that describe National Park visitation for that month. Your job is to list out which places were closed for the entire month."
        print(prompt)
processComments('AMIS', 2022)

COMMENTS:
['Spur 406 counters is still out of service, Black Brush remains closed, San Pedro counter was broken. Campground counts are undergoing a new accounting process.']

------------------------------------------------------------

INSTRUCTIONS:
Listed Above are comments over the year that describe National Park visitation for that month. Your job is to list out which places were closed for the entire month.
COMMENTS:
['Spur 406 remains on a work order to be replaced. Black Brush is closed. San Pedro counter is on a work order to be repaired. Campground counts are on a new accounting system.']

------------------------------------------------------------

INSTRUCTIONS:
Listed Above are comments over the year that describe National Park visitation for that month. Your job is to list out which places were closed for the entire month.
COMMENTS:
['Black Brush closed.']

------------------------------------------------------------

INSTRUCTIONS:
Listed Above are comments over the year 

In [71]:
getComments('BUFF', 2024)['Comments'].values

array(['USED AVG FOR OZARK CG RD, LOST VALLEY TRAIL, GILBERT, HWY 65, WOOLUM, MAUMEE SOUTH, BUFFALO POINT RD, AND SPRING CREEK  Concessions are closed due to winter time',
       'The following concessioners were closed still Buffalo River canoes, Buffalo Camping and Canoes, Silver hill, Crockett, and Dirst.  5 year avg used for camping at Spring Creek, Gilbert, Hwy 65, Woolum and South Maumee.  5 year avg used for Lost Valley and Hemmed in Hollow trails.',
       'Used avg for Camp Orr CG  Silver Hill canoe was closed.  Avg for Gilbert, Hwy 65 bridge, Woolum, and S Maumee'],
      dtype=object)

In [27]:
print(formatComments('WORI', 2020))

January:
M'Clintock House is Closed for the winter. Stanton House is Closed January and February.  Park is closed Mondays and Tuesdays to the public. Park closed New Year's Day. VC Exhibit Area Tour used to record stats for 30 minute programs at 11:30 and 3:30 in January and February in the Guntzel Theater about Elizabeth Cady Stanton and other early women's movement figures while the Stanton House is closed.   2 Jan-Back door closed to add ADA compliant buttons, 11:30 and 3:30 talks in theater cancelled due to staffing, sunny and visitors were hot in theater 3 Jan-Back door closed. 8 Jan-Back door closed. 9 Jan-Working on front and back doors to add ADA buttons. 17 Jan-Women's March Events in Auburn 18 Jan-Women's March in Seneca Falls, Rally 12:30-1:30; Visitation high for season at 463; Park Closed at 4 pm due to weather 19 Jan-Women's March Events SF Presbyterian Church 22 Jan-Back doors closed.  24 Jan-2 VIP's came in to help cover interpretation staff emergency absence.  Off Site